#  Classificação com NaiveBayes com Spark usando PySpark



## Inicializando o PySpark

In [49]:
import findspark
import pyspark
from pyspark.sql import SparkSession

# Faz a Interafce entre o Spark e o Jupyter Notebook
findspark.init()

# Inicializando uma Sessão no Spark
spark = SparkSession.builder.appName("NaiveBayes").getOrCreate()

### Hiper Parâmetros

> + **modelType**: Tipo de modelo. multinomial, bernoulli e gaussian. (Padrão: multinomial



---

## Aplicando  Classificação com NaiveBayes no DataSet Churn



### Carregando o Cunjunto de Dados Churn

In [50]:
iris = spark.read.csv("../Material_do_Curso/iris.csv",
                             header=True, inferSchema=True, sep=",")
print(f"Quantidade de Registros do Dataset: {iris.count()}")
iris.show(5, truncate=False)

Quantidade de Registros do Dataset: 150
+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|class      |
+-----------+----------+-----------+----------+-----------+
|5.1        |3.5       |1.4        |0.2       |Iris-setosa|
|4.9        |3.0       |1.4        |0.2       |Iris-setosa|
|4.7        |3.2       |1.3        |0.2       |Iris-setosa|
|4.6        |3.1       |1.5        |0.2       |Iris-setosa|
|5.0        |3.6       |1.4        |0.2       |Iris-setosa|
+-----------+----------+-----------+----------+-----------+
only showing top 5 rows



### Escolha das Variáveis Independentes e Dependente para treinamento do Modelo dados do dataset.

### Importação do Módulo do PySpark Para o Pré-Processamento dos Dados

In [51]:
from pyspark.ml.feature import RFormula

In [52]:
# Colocando os dados no Formato de dataframe do Spark usando o RFormula
rformula = RFormula(formula="class ~ .",
                    featuresCol="independente",
                    labelCol="dependente")

iris_rf = rformula.fit(iris).transform(iris)
iris_rf.select("independente", "dependente").show(5, truncate=False)

+-----------------+----------+
|independente     |dependente|
+-----------------+----------+
|[5.1,3.5,1.4,0.2]|0.0       |
|[4.9,3.0,1.4,0.2]|0.0       |
|[4.7,3.2,1.3,0.2]|0.0       |
|[4.6,3.1,1.5,0.2]|0.0       |
|[5.0,3.6,1.4,0.2]|0.0       |
+-----------------+----------+
only showing top 5 rows



### Separando os Dados entre conjunto de Treino e Teste

O Conjunto de dados serão separados entre treino e teste, sendo que **80%** será para treinar o modelo e **20%** para testar o modelo.

In [53]:
iris_train, iris_test = iris_rf.randomSplit([0.8, 0.2])
print(f"Quantidade de Dados de Treino: {iris_train.count()}")
print(f"Quantidade de Dados de Teste: {iris_test.count()}")

Quantidade de Dados de Treino: 121
Quantidade de Dados de Teste: 29


---

### Importação do Módulo do PySpark Para Criação do Modelo de Classificação com NaiveBayes

In [54]:
from pyspark.ml.classification import NaiveBayes

#### Instanciando Objeto e criando o Modelo

In [55]:
# Instanciando o objeto LinearRegression
obj_nb = NaiveBayes(smoothing=1.0, modelType="multinomial",
                    featuresCol="independente", labelCol="dependente")

# Criando o Modelo
model_nb = obj_nb.fit(iris_train)

#### Realizando Presição com o Modelo Criado

In [56]:
previsao_test = model_nb.transform(iris_test)
previsao_test.select("dependente", "prediction", "probability", "rawPrediction").show(5, truncate=True)

+----------+----------+--------------------+--------------------+
|dependente|prediction|         probability|       rawPrediction|
+----------+----------+--------------------+--------------------+
|       0.0|       0.0|[0.68501211319680...|[-11.888659407883...|
|       0.0|       0.0|[0.65986989861165...|[-11.764584095230...|
|       0.0|       0.0|[0.65080073087098...|[-11.727133339374...|
|       1.0|       1.0|[0.10631602380147...|[-17.197535354296...|
|       0.0|       0.0|[0.70063580375075...|[-11.244481911778...|
+----------+----------+--------------------+--------------------+
only showing top 5 rows



### Avaliando o Modelo

In [57]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [58]:
avaliar = MulticlassClassificationEvaluator(labelCol="dependente",
                                            predictionCol="prediction",
                                            metricName="accuracy")

acuracia = avaliar.evaluate(previsao_test)

print(f"Acurácia: {acuracia}")

Acurácia: 0.9310344827586207


---